Configurações Iniciais

In [1]:
modelSizes = [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,None]
_random_state = 1

Ler arquivo .CSV

In [2]:
import numpy as np
def convertNominalClassToNumber(class_name):
    #objective, negative, objective-OR-neutral, positive
    if(class_name == "objective"):
        return 0
    elif(class_name == "negative"):
        return 1
    elif(class_name == "objective-OR-neutral"):
        return 2
    elif(class_name == "positive"):
        return 3
    elif(class_name == "neutral"):
        return 4
    else:
        return 5

    
tweets = open('tweeti-b.dist.csv').read()
x = []
y = []
for line in tweets.split('\n'):
    try:
        tabs = line.split('\t')
    except:
        pass
    else:
        if(tabs[0] != ''):
            x.append(tabs[3])
            y.append(convertNominalClassToNumber(tabs[2][1:-1]))


Pré-processamento das frases

In [3]:
import re
from nltk.corpus import stopwords
def text_to_words ( text, link = True ):
    # Function to convert a text to a string of words
    # The input is a single string, and
    # the output is a single preprocessed string
    #
    # 1. Remove HTML
    # review_text = BeautifulSoup(raw_review).get_text()
    #
    # 2. Remove links
    if link:
        words = text.lower().split()
        nonlink_words = [w for w in words if not w[:3] == "htt"]
        nonlink_phrase = " ".join(nonlink_words)
    else:
        nonlink_phrase = text

    # 2. Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", nonlink_phrase)
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))
    #
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    #
    # 6. Join the words back into one string separated by space,
    # and return the result.
    return( " ".join( meaningful_words ))

non_normalized_x = x
x = []
for sentence in non_normalized_x:
    x.append(text_to_words(sentence))

Criando Modelo BoW (TF-IDF)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

models = []
model_test_train = []
model_not_splitted = []
for index, size in enumerate(modelSizes):
    models.append([])
    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    models[index].append(TfidfVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = size))

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    models[index].append(models[index][0].fit_transform(x))

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    models[index].append(models[index][1].toarray())
    
x = []
for model in models:
    x.append(model[2])

Função para obter o balanceamento

In [5]:
def sortFunction(y):
    return y[0]

def getBalancement(y, justClass=False):
    copy_y = []
    balancement = []
    for i in range(len(y)):
        copy_y.append(y[i])
    while True:
        try:
            copy_y[0]
        except:
            break
        else:
            len_class = 0
            _class = copy_y[0]
            new_y = []
            for i, res in enumerate(copy_y):
                if res == _class:
                    len_class += 1
                else:
                    new_y.append(res)
            if justClass:    
                balancement.append(_class)
            else:
                balancement.append((_class,len_class))
            copy_y = new_y
    if justClass:
        balancement.sort()
    else:
        balancement = sorted(balancement, key=sortFunction)
    return balancement 

Função para aplicar balanceamento, fazer teste com crossvalidation e retornar F1

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFECV
import numpy

def skf_cv(x, y, classifier, balance=None, folds=10):
    skf = StratifiedKFold(n_splits=folds)
    foldsVec = []
    for train_fold,test_fold in skf.split(x, y):
        foldsVec.append([train_fold,test_fold])
    score = 0.0
    for index, fold in enumerate(foldsVec):
        x_train = []
        y_train = []
        for trainInstance in fold[0]:
            x_train.append(x[trainInstance])
            y_train.append(y[trainInstance])
        if balance != None:
            x_train, y_train = balance(x_train,y_train)
        print getBalancement(y_train)
        classifier.fit(x_train,y_train)
        x_test = []
        y_test = []
        for index, testInstance in enumerate(fold[1]):
            x_test.append(x[testInstance])
            y_test.append(y[testInstance])
        y_pred = classifier.predict(x_test)
        score = score + f1_score(y_test, y_pred, average='weighted')
    return score/folds

def rfecv_score(estimator, x, y):
    return f1_score(y,estimator.predict(x), average='weighted')

def skf_cv_rfecv(x, y, classifier, balance=None, folds=10, rfecv=True):
    skf = StratifiedKFold(n_splits=folds)
    foldsVec = []
    if rfecv:
        wropper = RFECV(classifier, n_jobs=-1, step=10, cv=5, scoring=rfecv_score) #lambda clf, x, y: f1_score(y, clf.predict(x), average='weighted'))
    for train_fold,test_fold in skf.split(x, y):
        foldsVec.append([train_fold,test_fold])
    score = 0.0
    for index, fold in enumerate(foldsVec):
        x_train = []
        y_train = []
        for trainInstance in fold[0]:
            x_train.append(x[trainInstance])
            y_train.append(y[trainInstance])
        if balance != None:
            x_train, y_train = balance(x_train,y_train)
        if rfecv:
            x_train = wropper.fit_transform(x_train,y_train)
        classifier.fit(x_train,y_train)
        x_test = []
        y_test = []
        for testInstance in fold[1]:
            x_test.append(x[testInstance])
            y_test.append(y[testInstance])
        if rfecv:
            x_test = wropper.transform(x_test)
        y_pred = classifier.predict(x_test)
        score = score + f1_score(y_test, y_pred, average='weighted')
    return score/folds

In [17]:
from sklearn.svm import LinearSVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

class Result:
    def __init__(self, value, estimator, balancementMethod, features, wropper):
        self.value = value
        self.est = estimator
        self.balancementMethod = balancementMethod
        self.features = features
        self.wropper = wropper
        self.printOption = True
        self.printString = str(value)
    def detalhedView(self):
        if self.printOption:
            self.printString = "[" + str(self.value) + "]" + " Feat:" + str(self.features) + " Estimator: " + self.est.__str__() +  " Balance: " + str(self.balancementMethod) + " Wropper: " + str(self.wropper)
        else:
            self.printString = str(self.value)
        self.printOption = not self.printOption
        
    def __repr__(self):
        return self.printString
    def __str__(self):
        return self.printString

def smoteBalancing(x, y): 
    smote = SMOTE(random_state=_random_state)
    for i in getBalancement(y):
        x,y = smote.fit_sample(x,y)
    return x,y
  
def adasynBalancing(x, y):
    ada = ADASYN(random_state=_random_state)
    for i in getBalancement(y):
        x,y = ada.fit_sample(x,y)
    return x,y

def randomOverBalancing(x, y):
    return RandomOverSampler(random_state=_random_state).fit_sample(x ,y)


imb_functions = [adasynBalancing]
estimators = [LinearSVC(random_state=_random_state),SGDClassifier(random_state=_random_state),DecisionTreeClassifier(random_state=_random_state)]



res = []

res.append([])
ft_res = []
_rfecv = False
print("w/o RFECV")
for x_feat in x:
    print("\t" + str(x_feat.shape[1]))
    imb_res = [] 
    for fn in imb_functions:
        if fn == None:
            fn_name = str(None)
        else:
            fn_name = fn.__name__
            
        print("\t\t" + fn_name)
        est_res = []
        for est in estimators:
            print("\t\t\t" + est.__class__.__name__)
            r = skf_cv_rfecv(x_feat,y,est,balance=fn,rfecv=_rfecv)
            print "\t\t\t\t" + str(r)
            est_res.append(Result(r,x_feat.shape[1],est.__class__.__name__,fn_name,_rfecv))
        imb_res.append(est_res)
    ft_res.append(imb_res)
res[0].append(ft_res)


# res.append([])
# ft_res = []
# _rfecv = True
# print("RFECV")
# for x_feat in x:
#     print("\t" + str(x_feat.shape[1]))
#     imb_res = [] 
#     for fn in imb_functions:
#         if fn == None:
#             fn_name = str(None)
#         else:
#             fn_name = fn.__name__
            
#         print("\t\t" + fn_name)
#         est_res = []
#         for est in estimators:
#             print("\t\t\t" + est.__class__.__name__)
#             r = skf_cv_rfecv(x_feat,y,est,balance=fn,rfecv=_rfecv)
#             print "\t\t\t\t" + str(r)
#             est_res.append(Result(r,x_feat.shape[1],est.__class__.__name__,fn_name,_rfecv))
#         imb_res.append(est_res)
#     ft_res.append(imb_res)
# res[1].append(ft_res)


w/o RFECV
	1000
		adasynBalancing
			LinearSVC
				0.374955734117
			SGDClassifier
				0.356711562669
			DecisionTreeClassifier
				0.351699434397
	2000
		adasynBalancing
			LinearSVC
				0.380717540816
			SGDClassifier
				0.378346097198
			DecisionTreeClassifier
				0.357546956283
	3000
		adasynBalancing
			LinearSVC
				0.380431830346
			SGDClassifier
				0.368741129972
			DecisionTreeClassifier
				0.356940242006
	4000
		adasynBalancing
			LinearSVC
				0.38107016295
			SGDClassifier
				0.377221195161
			DecisionTreeClassifier
				0.362688792655
	5000
		adasynBalancing
			LinearSVC
				0.380144122394
			SGDClassifier
				0.37382642013
			DecisionTreeClassifier
				0.356414155991
	6000
		adasynBalancing
			LinearSVC
				0.381741551013
			SGDClassifier
				0.368737261935
			DecisionTreeClassifier
				0.36155157256
	7000
		adasynBalancing
			LinearSVC
				0.381722858738
			SGDClassifier
				0.374628749153
			DecisionTreeClassifier
				0.36534879144
	8000
		adasynBalancing
			LinearSV

KeyboardInterrupt: 

In [16]:
for feat in res[0]:
    for b in feat:
        for alg in b:
            for a in alg:
                print a

0.384685232214
0.353144835796
0.361974329391
0.384737613839
0.376372884945
0.35407135075
0.382429115988
0.386484797172
0.350568383012
0.389729084613
0.362693713458
0.366278234633
0.384717305498
0.377818839994
0.348887471017
0.379610191691
0.383513940023
0.347435596158
0.387370857388
0.361800038209
0.37283833834
0.387339168606
0.382071936379
0.34920750416
0.381626823348
0.38354443407
0.363754716241
0.385049390791
0.367912927711
0.373117226148
0.388783138742
0.381203921864
0.357584447768
0.387304859746
0.383579627705
0.361836609361
0.386669754514
0.357667818911
0.365027072058
0.393855932945
0.381072073973
0.35561251031
0.389357895216
0.39445622412
0.366233448044
0.38650133427
0.36215226318
0.368130333045
0.392882549948
0.38280741192
0.350750452475
0.387572010838
0.387670305476
0.361022082909
0.383445546282
0.357069250567
0.368589572296
0.394042716411
0.381480149637
0.358182551581
0.39265954817
0.391643080681
0.358302971911
0.383954007815
0.361414564695
0.37652967715
0.390311231116
0.3806

In [18]:
#Teste RFECV 13979 features SVM

print skf_cv_rfecv(x[-1],y,LinearSVC(random_state=_random_state),balance=randomOverBalancing,rfecv=True)
print skf_cv_rfecv(x[-1],y,LinearSVC(random_state=_random_state),balance=smoteBalancing,rfecv=True)
print skf_cv_rfecv(x[-1],y,SGDClassifier(random_state=_random_state),balance=randomOverBalancing,rfecv=True)

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:111

0.393642616199


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:111

0.392693962655


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:111

0.383652501979
